Baseline model
==============

future incidence (`t+7`) == current incidence (`t`)

In [1]:
import sys
sys.path.insert(0, "/home/iheredia/ignacio/covid/covid-dl/src")
from paths import PATHS
from data.base import make_splits, get_data, single_X_y

In [2]:
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

In [3]:
df = get_data(trend=False)
splits = make_splits(df,
                     norm=False)
X, y = single_X_y(splits)

# Simple check
assert X.loc[('2020-09-15', 'Albacete'), 'incidence 7'] == y.loc[('2020-09-08', 'Albacete'), 'incidence 7 (t+7)']

/home/iheredia/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [7]:
def model_summary():
    
    print('\n# Metrics')
    metrics = {'mae': {}, 'mape': {}}
    
    for i in ['train', 'val', 'test']:
        y_t = splits[i]['y'].loc[:, 'incidence 7 (t+7)']
        y_p = splits[i]['X']['incidence 7']

        mae = np.abs(y_p - y_t)
        mape = np.abs( (y_p - y_t) / y_t)
        mape = mape.replace([np.inf, -np.inf], np.nan)
        
        metrics['mae'][i] = mae.mean()
        metrics['mape'][i] = mape.mean()

        print(f' - {i.capitalize()}')
        print(f'   Mean Absolute Error (in incidence): {mae.mean():.4f}')
        print(f'   Mean Absolute Percentage Error (in incidence): {mape.mean():.4f}')
        
    return metrics


def save_metrics(metrics, modelname):
    for m in metrics.keys():
        b = pd.DataFrame.from_dict(metrics[m], orient='index', columns=[modelname]).T
        a = pd.read_csv(f'single_step/{m}.csv', index_col=0)
        a = pd.concat([a[~a.index.isin(b.index)], b])
        a.to_csv(f'single_step/{m}.csv')

In [9]:
metrics = model_summary()
save_metrics(metrics, 'Baseline')


# Metrics
 - Train
   Mean Absolute Error (in incidence): 26.1197
   Mean Absolute Percentage Error (in incidence): 0.4659
 - Val
   Mean Absolute Error (in incidence): 59.5882
   Mean Absolute Percentage Error (in incidence): 0.2950
 - Test
   Mean Absolute Error (in incidence): 45.0234
   Mean Absolute Percentage Error (in incidence): 0.7371
